In [1]:
""" 
1. Использовать dataset MovieLens
2. Построить рекомендации ( предсказываем оценку)
на фичах:
■ TF IDF на тегах и жанрах
■ Средние оценки median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке
"""
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm  #  интеллектуальный индикатор выполнения - 
                                # просто оберните любую итерацию с помощью tqdm (iterable)

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline




In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# задача - мы должны понять, какую оценку пользователь поставит фильму (целевая переменная - rating)
# 1. Для этого мы изучим: 
#     1.1. фильмам каких жанров человек ставит оценки
#           (отфильтровать по пользователю, выделить фильмы, которым он поставил оценки, узнать жанры этих фильмов)
#     1.2. какие оценки ставит пользователь данным жанрам
#     1.3. как он размечает фильмы тегами 
#     1.4. расставим веса жанров и тегов методом tfidf 
# 2. соединяем таблицы фильмов и рейтингов, чтобы понимать, какому фильм какая оценка поставлена (для тестовой выборки)
# 3. сформируем линейную регрессию и сделаем прогноз
# 4. найдем ошибку RMSE

In [6]:
# movies_tags = movies.join(tags.set_index('movieId'), on='movieId')
movies_tags = movies.merge(tags, on='movieId')
movies_tags

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932
...,...,...,...,...,...,...
3678,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,star wars,1528934552
3679,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,anime,1537098582
3680,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,comedy,1537098587
3681,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,gintama,1537098603


In [7]:
movies_tags_ratings = movies_tags.merge(ratings.set_index('movieId'), on='userId')

movies_tags_ratings.head()

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1122227549
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.5,1122227343
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,5.0,1120568496
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1120568169


In [8]:
new_df = movies_tags_ratings.pop('timestamp_x')


In [9]:
movies_tags_ratings

,movieId,title,genres,userId,tag,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,1122227549
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.5,1122227343
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,5.0,1120568496
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,1120568169
...,...,...,...,...,...,...,...
4626861,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5,1515194979
4626862,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.0,1533519112
4626863,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,5.0,1529413865
4626864,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5,1533519140


In [10]:
movies_tags_ratings_ = movies_tags_ratings.pop('timestamp_y')

In [11]:
# Либо удалить столбцы можно было через команду del или так:
# movies_tags_ratings.drop(['timestamp_x', 'timestamp_y'], axis=1).head()

In [12]:
movies_tags_ratings

,movieId,title,genres,userId,tag,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,5.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
...,...,...,...,...,...,...
4626861,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5
4626862,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.0
4626863,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,5.0
4626864,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5


In [13]:
# user_462 = movies_tags_ratings[movies_tags_ratings.userId==462]
# user_462.sample(10)

In [14]:
def change_string(s):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце символом '|', 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [15]:
# прогоним через функцию строки столбца жанров 
movie_genres = [change_string(g) for g in movies_tags_ratings.genres.values]

In [16]:
movies_tags_ratings['Movie_genres'] = movie_genres

In [34]:
movies_tags_ratings.head()

,movieId,title,genres,userId,tag,rating,Movie_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,Adventure Animation Children Comedy Fantasy
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,Adventure Animation Children Comedy Fantasy
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.5,Adventure Animation Children Comedy Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,5.0,Adventure Animation Children Comedy Fantasy
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,Adventure Animation Children Comedy Fantasy


In [93]:
# Объявим функцию для работы с тегами
def change_string_tag(s):
    """
    Функция принимает строку и возвращает список слов, разделенных в строке запятой, 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(s.replace(' ', '').replace('-', '').split(', '))

In [94]:
# прогоним через функцию строки столбца тегов
movie_tags = [change_string_tag(g) for g in movies_tags_ratings.tag.values]

In [95]:
movies_tags_ratings['Movie_tags'] = movie_tags

In [96]:
movies_tags_ratings.sample(5)

,movieId,title,genres,userId,tag,rating,Movie_genres,Movie_tags
2578485,6867,"Station Agent, The (2003)",Comedy|Drama,474,trains,4.5,Comedy Drama,trains
3364135,6058,Final Destination 2 (2003),Horror|Thriller,62,sequel,4.5,Horror Thriller,sequel
1528291,2939,Niagara (1953),Drama|Thriller,474,In Netflix queue,4.0,Drama Thriller,InNetflixqueue
1402073,2565,"King and I, The (1956)",Drama|Musical|Romance,474,Siam,2.5,Drama Musical Romance,Siam
2916153,8645,"Maria Full of Grace (Maria, Llena eres de grac...",Crime|Drama,474,drugs,3.5,Crime Drama,drugs


In [44]:
# обучим модель для понимания, каким жанром принадлежит каждый их фильмов 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)
X_train_counts.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [46]:
# обучим модель, чтобы придать жанрам различные веса 
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [47]:
# реализовав метод, видим, что вес фильмов изменился в зависимости от его жанров (чем реже жанр, тем выше вес) 
X_train_tfidf.toarray()

array([[0.        , 0.3812444 , 0.53467319, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.3812444 , 0.53467319, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.3812444 , 0.53467319, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [48]:
# обучим модель для понимания, какими тегами описывается каждый из фильмов 
# для экономии памяти не будем трансформировать разреженную матрицу в обычный массив 
X_train_tags = count_vect.fit_transform(movie_tags)

In [49]:
# уникальных значений в столбце тегов много 
movies_tags_ratings.tag.nunique()

1589

In [50]:
# некоторе теги встречаются заметно чаще остальных: это значит, что метод tfidf способен разнести веса от высоких к низким
movies_tags_ratings['tag'].value_counts() 

In Netflix queue    276148
Disney               44669
religion             43360
crime                37818
superhero            30262
                     ...  
old                     24
moldy                   24
jackie chan             22
kung fu                 22
black hole              21
Name: tag, Length: 1589, dtype: int64

In [51]:
# посмотрим, есть ли пустые значения столбца тегов: нет 
movies_tags_ratings['tag'].isnull().any()

False

In [52]:
# обучим модель, чтобы придать тегам различные веса 
tfidf_transformer = TfidfTransformer()
X_train_tfidf_tags = tfidf_transformer.fit_transform(X_train_tags)

In [141]:
# Чтобы сократить количество тегов (их, как видим, много), 
# низкочастотные теги заменим на значение other. Сделаем это так:

# Создадим отдельный датафрейм с value_counts
# Отберем из него элементы с нужной частотой
# Используем индекс этих элементов - именно в индексе будет список нужных тегов (а в значениях там - число повторений тегов)
# Отфильтруем колонку с тегами по вхождению элементов столбца в список выше найденных редких тегов
# Присвоим нафильтрованным строкам в колонку с тегами наш новый тег

vc = movies_tags_ratings['Movie_tags'].value_counts()
movies_tags_ratings.loc[movies_tags_ratings['Movie_tags'].isin(vc[vc < 1500].index), 'Movie_tags'] = 'other'

In [142]:
# Проверим
rslt_df = movies_tags_ratings.loc[movies_tags_ratings['Movie_tags'] == 'other']
rslt_df.head()

,movieId,title,genres,userId,tag,rating,Movie_genres,Movie_tags
112,552,"Three Musketeers, The (1993)",Action|Adventure|Comedy|Romance,336,knights,4.0,Action Adventure Comedy Romance,other
113,552,"Three Musketeers, The (1993)",Action|Adventure|Comedy|Romance,336,knights,4.0,Action Adventure Comedy Romance,other
114,552,"Three Musketeers, The (1993)",Action|Adventure|Comedy|Romance,336,knights,4.5,Action Adventure Comedy Romance,other
115,552,"Three Musketeers, The (1993)",Action|Adventure|Comedy|Romance,336,knights,5.0,Action Adventure Comedy Romance,other
116,552,"Three Musketeers, The (1993)",Action|Adventure|Comedy|Romance,336,knights,4.0,Action Adventure Comedy Romance,other


In [145]:
# проверим количество уникальных значений тегов теперь; 
# при необходимости откалибруем фильтр частотности 2-мя ячейками выше
movies_tags_ratings.Movie_tags.nunique()

878

In [146]:
# Другие возможности для экономии пространства:
# 1) заменить все редкие теги одним значением (например, other) можно разными способами:

# DataFrame['column_name'] = numpy.where(condition, new_value, DataFrame.column_name)
# еще можно так:
# movies_tags_ratings['tag'].replace(movies_tags_ratings['tag'].value_counts()<100, 'other')

# 2) параметрами ограничить число фич, создаваемых нашими трансформерами
# 3) мы оставили матрицу разреженной. В sklearn есть модели, которые умеют с ними работать, например, Naive Bayes
# 4) предварительно можно понизить размерность через TruncatedSVD, PCA

# from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD
# pca = TruncatedSVD(n_components = 1)
# XPCAreduced = pca.fit_transform(transpose(movies_tags_ratings))

In [149]:
# оставим в столбце Movie_tags только уникальные значения тегов

# movies_tags_ratings['Movie_tags'] = movies_tags_ratings.Movie_tags.unique()

movies_tags_ratings.Movie_tags.value_counts()

other             319332
InNetflixqueue    276148
Disney             44669
religion           43360
crime              37818
                   ...  
enigmatic           1540
melancholy          1540
heartbreaking       1540
romance             1520
ChristianBale       1504
Name: Movie_tags, Length: 878, dtype: int64

In [168]:
"""
Теперь необходимо:
1) для каждого фильма оставить только уникальные теги: а если для данного фильма 
разные пользователи ставили 5 раз одинаковый тег - мы оставляем его 1 раз, а для всех прочих меняем значение на other ? 
2) Снова сделаем по ним tf idf (реализовано)
3) получить датафрейм вида | название фильма | столбцы tf idf по жанрам | столбцы tf_idf по тегам  : с этим засада (см. далее)
Здесь задача каждому значению показателя отдать свою колонку, верно? Так ли я это выполнил в ячейке 196? Надо ли транспонировать, 
чтобы вытянуть столбцы в строки? Почему-то размер до и после транспонирования не меняется и не показывает наличие столбцов
4) Если все верно, затем я хочу полученный объект по тегам добавить к нашему датафрейму методом .insert() (ячейка 212): 
все ли так?
5) видимо, на завершающем этапе подготовки фрейма к обучению регрессией надо удалить столбцы тегов, жанров, X_tfidf_genres,
X_tfidf_tags и Movie_genres,Movie_tags - так норм? 

"""

# uniq = movies_tags_ratings.Movie_tags.unique()
# movies_tags_ratings.loc[movies_tags_ratings['Movie_tags'].isin(uniq)].head(15)

# movie_452 = movies_tags_ratings[movies_tags_ratings.movieId==152711].Movie_tags
# movie_452

In [169]:
X_train_tags_too = count_vect.fit_transform(movie_tags)
tfidf_transformer = TfidfTransformer()
X_train_tfidf_tags = tfidf_transformer.fit_transform(X_train_tags_too)

In [196]:
movies_tags_ratings.head()
tf_idf_model_genres = np.asarray(movies_tags_ratings['X_tfidf_genres'])
tf_idf_model_tags = np.asarray(movies_tags_ratings['X_tfidf_tags'])

In [206]:
# транспонируем объекты, чтобы вытянуть их в строку 
tf_idf_model_genres = np.transpose(tf_idf_model_genres)
tf_idf_model_tags = np.transpose(tf_idf_model_tags)

In [212]:
# movies_tags_ratings.insert(-1, "tf_idf_model_genres", tf_idf_model_genres)

In [208]:

# 1. Список по всем фильмам
# | Крепкий орешек | Action Drama Triller | interesing brus very_good |
# 2. На этом списке учите 2 TfidfTransfomer’а
# - один на столбце с жанрами
# - на столбец с тегами
# 3. Тегов много. Поэтому все низкочасттотные теги заменяете на один other
# 4. Оставляете для каждого фильма только уникальные теги
# 5. Снова делаете делаете по ним tf idf
# 6. В итоге получаете df вида
# | название фильма | столбцы tf idf по жанрам | столбцы tf_idf по тегам
# 7.  добавляете оценки, фильтруете и оставлете только фильмы от конркетного пользвателя
# 8. строите модель

In [209]:
# Удалим столбцы жанров и тегов с тем, чтобы следом добавить признаки, созданные трансформерами 
# user_462.drop(['genres', 'tag'], axis=1).head()